# GPT 언어 모델

GPT가 공개되었을 당시에는 굉장히 주목을 받았습니다.

자연어 문장을 분류해주는 아주 성능 좋은 디코더!
적은 양의 데이터로도 좋은 성능 발휘 가능
다양한 자연어 태스크에서 최고 성능 달성
등등 굉장히 많은 장점들을 가지고 있었습니다.

하지만 당연히 아직 해결하지 못한 문제들 또한 남아 있었습니다.

지도학습이 필수적 (라벨이 있는 데이터가 필요!)
특정 태스크에 맞게 학습된 모델은 다른 태스크에서는 사용 불가

# GPT-2 사전학습해보기

In [ ]:
!pip install transformers

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
!mkdir my_data
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" -o my_data/wiki_20190620_small.txt

path = "/content/my_data/wiki_20190620_small.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1323k  100 1323k    0     0  1726k      0 --:--:-- --:--:-- --:--:-- 1726k


In [ ]:
from tokenizers import SentencePieceBPETokenizer
from tokenizers.normalizers import BertNormalizer

tokenizer = SentencePieceBPETokenizer()

tokenizer._tokenizer.normalizer = BertNormalizer(clean_text=True,
handle_chinese_chars=False,
lowercase=False)

tokenizer.train(
    path,
    vocab_size=10000,
    special_tokens=[
        "<s>", # 문장의 시작
        "<pad>", # 패딩 토큰
        "</s>", # 문장의 끝
        "<unk>", # 사전에 없는 토큰
    ],
)

In [ ]:
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").ids)
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").tokens)
print(tokenizer.decode(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids, skip_special_tokens=True))
# SentencePiece를 사용하면, 나중에 decoding 과정에서 '_' 만 ' '로 replace해주면 띄어쓰기 복원이 가능해집니다.

[1005, 577, 6613, 1303, 1041, 2071, 1136, 594, 1033]
['▁이', '순', '신은', '▁조선', '▁중', '기의', '▁무', '신', '이다.']
이순신은 조선 중기의 무신이다.


In [ ]:
tokenizer.save_model('.')

['./vocab.json', './merges.txt']

In [ ]:
tokenizer= SentencePieceBPETokenizer.from_file(vocab_filename='vocab.json', meges_filename="merges.txt")

NameError: name 'vocab_filename' is not defined

In [ ]:
print(tokenizer.encode("이순신은 조선 중기의 무신이다."))
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").ids)
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").tokens)
print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").tokens)
print(tokenizer.decode(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids, skip_special_tokens=True))


In [ ]:
tokenizer.add_special_tokens(["<s>", "</s>", "<unk>", "<pad>", "<shkim>"])
tokenizer.pad_token_id = tokenizer.token_to_id("<pad>")
tokenizer.unk_token_id = tokenizer.token_to_id("<unk>")
tokenizer.bos_token_id = tokenizer.token_to_id("<bos>")
tokenizer.eos_token_id = tokenizer.token_to_id("<eos>")

print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids)
print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").tokens)
print(tokenizer.decode(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids, skip_special_tokens=True))

In [ ]:
from transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(
  vocab_size=tokenizer.get_vocab_size(),
  bos_token_id=tokenizer.token_to_id("<s>"),
  eos_token_id=tokenizer.token_to_id("</s>"),
)

model = GPT2LMHeadModel(config)

In [ ]:
model.num_parameters()

In [ ]:
import json
import os
import pickle
import random
import time
import warnings
from typing import Dict, List, Optional

import torch
from torch.utils.data.dataset import Dataset

from filelock import FileLock

from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.utils import logging

In [ ]:
logger = logging.get_logger(__name__)

class TextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        block_size = block_size - tokenizer.num_special_tokens_to_add(is_pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            "cached_lm_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )

            else:
                logger.info(f"Creating features from dataset file at {directory}")
                # 여기서부터 본격적으로 데이터셋을 만들기 시작합니다.
                self.examples = []
                text = ""
                with open(file_path, encoding="utf-8") as f: # file_path가 path 말하는거임
                    lines = f.readlines()
                    for line in lines:
                        line = line.strip()
                        line = "<s>"+line+"</s>" # 학습 데이터 앞 뒤에 문장 구분 기호를 추가해줍니다.
                        text += line    # 'text' 객체에 모든 학습 데이터를 다 합쳐버립니다 :-)
                tokenized_text = tokenizer.encode(text).ids

                # 모델의 최대 sequence length만큼 데이터를 잘라서 저장합니다.
                for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                    self.examples.append(
                        tokenized_text[i : i + block_size]
                    )
                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should look for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=path,
    block_size=128,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(    # GPT는 생성모델이기 때문에 [MASK] 가 필요 없습니다 :-)
    tokenizer=tokenizer, mlm=False,)

In [ ]:
print(dataset[0])

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_output',
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_device_train_batch_size=64, # 512:32  # 128:64
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100

)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
USE_GPU = 1
device= torch.device('cuda' if (torch.cuda.is_available() and USE_GPU) else 'cpu')

In [ ]:
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode("<s>이순신", add_special_tokens=True).ids).unsqueeze(0).to('cuda')

output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=100, num_return_sequences=3)
for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print("GENERATED SEQUENCE : {0}".format(tokenizer.decode(generated_sequence, skip_special_tokens=True)))

# 자연어 생성 직접해보기

In [ ]:
# KoGPT-2모델 사용
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.12
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Need to get 7,420 kB of archives.
After this operation, 6,051 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu jammy/main amd64 git-lfs amd64 3.5.1 [7,420 kB]
Fetched 7,420 kB in 1s (12.2 MB/s)
(Reading database ... 121757 files and directories c

In [ ]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
config.pad_token_id = tokenizer.token_to_id('<pad>')
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
model.eval()
tokenizer.add_special_tokens(["<s>", "</s>"])

In [ ]:
def tokenizing(text):
    return torch.tensor(tokenizer.encode('<s> '+text, add_special_tokens=False).ids).unsqueeze(0).to('cuda')

input_ids = tokenizing("이순신은 조선 중기의 무신이다.")

# generate text until the output length (which includes the context length) reaches 100
# 생성 모델은 generate 함수를 통해 다음 token을 생성해낼 수 있습니다.
greedy_output = model.generate(input_ids, max_length=100)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output.tolist()[0], skip_special_tokens=True))

In [ ]:
# beam_sesarch
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output.tolist()[0], skip_special_tokens=True))

In [ ]:
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

In [ ]:
pritn('Output:\n' + 100*'-')
print(tokenizer.decode(beam_output.tolist()[0], skip_special_tokens=True))

In [ ]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True
)

In [ ]:
pritn('Output:\n' + 100*'-')
for i, beam_output in enumerate(beam_outputs):
  print('{}: {}'.format(i, tokenizer.decode(beam_output.tolist(), skip_special_tokens=True)))

In [ ]:
# sampling
sample_output = model.generate(
    input_ids,
    do_sample=True, # 완전 random sampling
    max_length=50,
    top_k=0 # w/o top_k 추출
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

In [ ]:
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0,
    temperature=0.7
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

In [ ]:
# Top-K Sampling
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

In [ ]:
# Top-p(nucleus) Sampling
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.92,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

# Few-shot 학습 및 koGPT 사용해보기

In [ ]:
!pip install transformers
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

In [ ]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
config.pad_token_id = tokenizer.token_to_id('<pad>')
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')



In [ ]:
tokenized_text = tokenizer.encode('이순신은 조선 중기의 무신이다.', add_special_tokens=True)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

In [ ]:
tokenizer.add_special_tokens(["<s>", "</s>"])

In [ ]:
##############뒤에짤림
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode('이순신은', add_special_tokens=True).ids).unsqueeze(0).)

output~~~

In [ ]:
def get_gpt_output(input_sent):
    input_ids = torch.tensor(tokenizer.encode(input_sent, add_special_tokens=True).ids).unsqueeze(0).to('cuda')
    sample_outputs = model.generate(
        input_ids,
        do_sample=True,
        max_length=512,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.token_to_id("</s>"),
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    generated_sequence = sample_outputs[0].tolist()
    return tokenizer.decode(generated_sequence, skip_special_tokens=True)


In [ ]:
get_gpt_output('<s>이순신은')

In [ ]:
get_gpt_output("<s>철수 : 영희야 안녕!</s><s>영희 : ")

In [ ]:
get_gpt_output("<s>철수 : 영희야 안녕!</s><s>영희 : 어! 철수야! 오랜만이다!</s><s>철수 : 그러게~ 잘 지냈어?</s><s>영희 : ")

In [ ]:
get_gpt_output("<s>본문 : 아.. 기분 진짜 짜증나네ㅡㅡ</s><s>감정 : 분노</s><s>본문 : 와!! 진짜 너무 좋아!!</s><s>감정 : ")

In [ ]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 인공지능의 미래에 대해 어떻게 생각하십니까?</s>\
<s>답 : ")

In [ ]:
get_gpt_output("<s>한국어: 그 도로는 강과 평행으로 뻗어 있다.</s>\
<s>English: The road runs parallel to the river.</s>\
<s>한국어: 그 평행선들은 분기하는 것처럼 보인다.</s>\
<s>English: The parallel lines appear to diverge.</s>\
<s>한국어: 그 도로와 운하는 서로 평행하다.</s>\
<s>English: The road and the canal are parallel to each other.</s>\
<s>한국어: 평행한 은하계라는 개념은 이해하기가 힘들다.</s>\
<s>English: The idea of a parallel universe is hard to grasp.</s>\
<s>한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다.</s>\
<s>English: This tradition has no parallel in our culture.</s>\
<s>한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다.</s>\
<s>English: This is an achievement without parallel in modern times.</s>\
<s>한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다.</s>\
<s>English: It is possible to draw a parallel between their experience and ours.</s>\
<s>한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다.</s>\
<s>English: The new degree and the existing certificate courses would run in parallel.</s>\
<s>한국어: 이순신은 조선 중기의 무신이다.</s>\
<s>Englisth: ")

In [ ]:
# 한국어 언어모델 학습 및 다중 과제 튜닝
# koGPT-2 기반의 챗봇 실습
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs


In [ ]:
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2
!pip install transformers

In [ ]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')


In [ ]:
tokenized_text = tokenizer.encode('이순신은 조선 중기의 무신이다.', add_special_tokens=True)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

In [ ]:
##############뒤에짤림
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode('이순신은', add_special_tokens=True).ids).unsqueeze(0).)

output~~~

In [ ]:
# 데이터셋 준비
!git clone https://github.com/songys/Chatbot_data.git

import pandas as pd
data = pd.read_csv('/content/Chatbot_data/ChatbotData.csv')

data.head(10)


In [ ]:
added_special_token_num = tokenizer.add_special_tokens(['<s>', '</s>'])

print(added_special_token_num)

In [ ]:
pad_id = tokenizer.token_to_id("<pad>")
print(pad_id)
tokenizer.enable_padding(pad_id=pad_id, pad_token="<pad>")
tokenizer.enable_truncation(max_length=128)

In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, file_path):
        self.data = []
        self.file_path = file_path
        self.tokenizer = tokenizer

    def load_data(self):
        raw_data = pd.read_csv(self.file_path)
        train_data = '<s>'+raw_data['Q']+'</s>'+'<s>'+raw_data['A']+'</s>'
        #<s>안녕하세요</s><s> -> 네, 안녕하세요</s>
        tokenized_train_data = tokenizer.encode_batch(train_data)
        for single_data in tokenized_train_data:
            self.data.append(torch.tensor(single_data.ids).unsqueeze(0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        return item

train_dataset = ChatDataset(tokenizer=tokenizer, file_path='/content/Chatbot_data/ChatbotData.csv')
train_dataset.load_data()

In [ ]:
from torch.utils.data import DataLoader

data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)


In [ ]:
from transformers import AdamW

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-4, correct_bias=True)

epochs = 3

avg_loss = (0.0, 0.0)
for epoch in range(epochs):
    count=0
    for data in data_loader:
        optimizer.zero_grad()
        data = data.transpose(1,0)
        data = data.to('cuda')
        model = model.to('cuda')

        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss = loss.to('cuda')
        loss.backward()
        avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
        optimizer.step()
        if count % 200 == 0:
            print('epoch no.{0}  train ({1}/{2})  loss = {3:.5f}  avg_loss = {4:.5f}' . format(epoch, count, len(data_loader), loss, avg_loss[0] / avg_loss[1]))
        count += 1

In [ ]:
torch.save(model.state_dict(), 'chitchat_model.bin')

In [ ]:
def encoding(text):
    text = '<s>'+text+'</s><s>'
    return torch.tensor(tokenizer.encode(text).ids).unsqueeze(0).to('cuda')

def decoding(ids):
    return tokenizer.decode_batch(ids)

tokenizer.no_padding()
tokenizer.no_truncation()

e_s = tokenizer.token_to_id('</s>')
unk = tokenizer.token_to_id('<unk>')

In [ ]:
def get_answer(input_sent):
    input_ids = encoding(input_sent)

    sample_outputs = model.generate(
        input_ids,
        num_return_sequences=5,
        do_sample=True,
        max_length=128,
        top_k=50,
        top_p=0.95,
        eos_token_id=e_s,
        early_stopping=True,
        bad_words_ids=[[unk]]
    )

    decoded_result = decoding(sample_outputs.tolist())
    for result in decoded_result:
        print(result)

In [ ]:
get_answer('안녕?')

In [ ]:
get_answer('만나서 반가워.')

In [ ]:
get_answer('인공지능의 미래에 대해 어떻게 생각하세요?')

# 트랜스포머 기반 언어 모델 연대기

# 멀티 모달